In [1]:
#pip install networkx[default]

In [1]:
import json
import networkx as nx
import numpy as np
import pandas as pd

from sklearn.cluster import AgglomerativeClustering

In [2]:
# csv paths
nutrient_matrix_data_p = r"../../data/Nutrients_Branded_Foods_2018/"

nutrient_matrix_csv_p = nutrient_matrix_data_p + "nutrients_matrix.csv.gz"

In [3]:
#parameters to save the linkage matrix
metric = 'euclidean'

linkage = 'single'

fname = nutrient_matrix_data_p + 'hcluster_nutrients_' + metric + '_' + linkage + '.npy'

In [10]:
#read data
nutrients_matrix = pd.read_csv(nutrient_matrix_csv_p)
#set ndb no as index
nutrients_matrix.set_index("NDB_No", inplace = True)
#create a list of indices where all entries are zero
nutrients_zero_rows = nutrients_matrix.loc[(nutrients_matrix==0).all(axis=1)].index
#filter out indicies where all entires are zero
nutrients_matrix = nutrients_matrix[~nutrients_matrix.index.isin(nutrients_zero_rows)]
print(nutrients_matrix.shape)

(234247, 94)


In [6]:
sample = nutrients_matrix.sample(frac = 0.1, random_state = 1)

In [7]:
clusters, _ = sample.shape
clusters = round(clusters*0.02)

In [8]:
agglo_cluster = AgglomerativeClustering(n_clusters = clusters, metric=metric, linkage = linkage, compute_full_tree = False).fit(sample)

In [9]:
agglo_cluster.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [10]:
linkage_matrix = agglo_cluster.children_

In [12]:
linkage_matrix.shape

(23424, 2)

#### Create a json object from the linkage matrix

In [4]:
linkage_matrix = np.load(fname, allow_pickle=True)

In [5]:
linkage_matrix

array([[ 98077, 189065],
       [ 49193, 157413],
       [ 47853,  93762],
       ...,
       [468489,  77617],
       [468490, 218766],
       [468491, 201900]])

In [34]:
from heapq import heapify, heappop, heappush, heappushpop

In [35]:
n_clusters = round(nutrients_matrix.shape[0]*0.02)
children = linkage_matrix
n_leaves = nutrients_matrix.shape[0]

In [43]:
n_clusters

4685

In [136]:
nodes = [-(max(linkage_matrix[-1]) + 1)]

In [135]:
nodes

[-468487, -201900, -218766, -77617, -176764, -93184]

In [128]:
these_children = children[-nodes[0] - n_leaves]

In [129]:
-these_children

array([-468489,  -77617])

In [130]:
heappush(nodes, -these_children[0])
nodes

[-468490, -468489, -218766, -201900]

In [131]:
heappushpop(nodes, -these_children[1])
nodes

[-468489, -201900, -218766, -77617]

In [137]:
for _ in range(n_clusters - 1):
    # As we have a heap, nodes[0] is the smallest element
    these_children = children[-nodes[0] - n_leaves]
    # Insert the 2 children and remove the largest node
    heappush(nodes, -these_children[0])
    heappushpop(nodes, -these_children[1])

In [139]:
nodes[0:5]

[-463808, -463807, -463802, -463804, -463799]

In [140]:
len(nodes)

4685

In [44]:
label = np.zeros(n_leaves, dtype=np.intp)

In [52]:
label[[0,1,2,3]]=1

In [54]:
label[0:5]

array([1, 1, 1, 1, 0], dtype=int64)

In [142]:
node = -nodes[0]

In [143]:
node

463808

In [144]:
ind = [node]
descendent = []

# It is actually faster to do the accounting of the number of
# elements is the list ourselves: len is a lengthy operation on a
# chained list
#cdef cnp.intp_t i, 
n_indices = 1

while n_indices:
    i = ind.pop()
    if i < n_leaves:
        descendent.append(i)
        n_indices -= 1
    else:
        ind.extend(children[i - n_leaves])
        n_indices += 1
descendent

[13206, 13205, 214823, 1103]

In [17]:
DG = nx.DiGraph()

In [18]:
num_rows, _ = linkage_matrix.shape
i = 0
# loop linked matrix convert to dictionary
for row in linkage_matrix:
    i += 1
    DG.add_edge(row[0], i + num_rows)
    DG.add_edge(row[1], i + num_rows)

In [19]:
g = list(DG.nodes)

In [18]:
print(nx.shortest_path(DG, source=20000, target = max(g)))

[20000, 37327, 37334, 37336, 37350, 37355, 37361, 37365, 37367, 37370, 37384, 37386, 37399, 37404, 37405, 37406, 37412, 37414, 37422, 37423, 37429, 37432, 37437, 37445, 37446, 37447, 37449, 37454, 37460, 37462, 37465, 37466, 37469, 37470, 37472, 37477, 37484, 37489, 37490, 37497, 37499, 37509, 37515, 37516, 37520, 37522, 37523, 37524, 37526, 37527, 37528, 37533, 37540, 37543, 37545, 37547, 37548, 37550, 37551, 37554, 37557, 37559, 37562, 37563, 37564, 37566, 37570, 37571, 37575, 37589, 37590, 37593, 37597, 37599, 37602, 37605, 37606, 37609, 37615, 37617, 37620, 37622, 37623, 37627, 37631, 37637, 37638, 37643, 37646, 37650, 37654, 37655, 37662, 37663, 37664, 37666, 37668, 37670, 37675, 37678, 37679, 37682, 37685, 37688, 37689, 37697, 37698, 37700, 37701, 37705, 37706, 37708, 37710, 37722, 37723, 37724, 37731, 37733, 37735, 37736, 37737, 37739, 37743, 37753, 37758, 37760, 37762, 37763, 37767, 37768, 37771, 37774, 37780, 37781, 37782, 37784, 37790, 37800, 37802, 37805, 37806, 37809, 37813

In [ ]:
assert not test_recursion(lambda: map(factorial_iterative, range(5)))

Build the json object with recursion.
Inspiration:

https://stackoverflow.com/questions/65462806/how-to-parse-data-from-scikitlearn-agglomerative-clustering

In [17]:
def create_tree(linked, leaf_labels):
    ## inner func to recurvise-ly walk the linkage matrix
    def recurTree(tree):
        k = tree['clust']
        ## no children for this node
        if k not in inter:
            tree['nbd_no'] = int(leaf_labels[k])
            del tree["clust"]
            del tree["children"]
            return
        for n in inter[k]:
            ## build child nodes
            node = {
                "clust": int(n),
                "parent": k,
                "children": []
            }
            ## add to children
            tree['children'].append(node)
            ## next recursion
            recurTree(node)      
    
    num_rows, _ = linked.shape
    inter = {}
    i = 0
    # loop linked matrix convert to dictionary
    for row in linked:
        i += 1
        inter[i + num_rows] = [row[0],row[1]]

    # start tree data structure
    tree = {
        "clust": i + num_rows,
        #"parent": None,
        "children": []
    }
    # start recursion
    recurTree(tree);
    return tree

In [ ]:
d = create_tree(linkage_matrix, nutrients_matrix.index)

In [41]:
json.dump(d, open(nutrient_matrix_data_p + 'hcluster_nutrients_' + metric + '_' + linkage + 'json', "w"))